In [ ]:
import logging
import json


from pymongo import MongoClient
from importlib import resources
import os
import re
import datetime
import glob

from codex.database.vasp import generate_json as vasp_generate_json
from codex.database.espresso import run_helpdoc
from codex.database.vasp import generate_json as espresso_generate_json
from codex.database.utils import get_database

logging.basicConfig(level=logging.WARNING)

In [ ]:
client = MongoClient('localhost', 27017)

In [ ]:
from codex.models.vasp_codex import VaspCodex
from codex.models.espresso_codex import EspressoCodex
from codex.api.schemas import CodexEntrySchema

codex1 = VaspCodex('example/vasp/POSCAR_Fe3O4.vasp', 'latest', client)
codex2 = EspressoCodex('example/espresso/bi4te3_bandsx.in', 'latest', client)
#client.drop_database('codex')
result = client['codex']['entries'].insert_many(CodexEntrySchema().dump([codex1, codex2], many=True))
#print(result.inserted_id)

In [ ]:
result.inserted_ids[0]

In [55]:
from codex.api.schemas import CodexEntryQueryArgsSchema
params = {'filetype': 'INCAR', 'code': 'vasp'}
#params = {'_id': ['e44e5a84f8034db1a9fab5b391c2cd34', 'dff217b9190e4d1db5d118d09d0463f7']}
#print(CodexEntryQueryArgsSchema().dump(params))
x = requests.get(URL, params=params).json()
for xx in x:
    print(xx['_id'])

3954583e-94ba-41df-90f3-86d673ce5189
83e594ee-ad85-4c81-b72f-ffa53d8ce9a4
dff217b9190e4d1db5d118d09d0463f7
4b364556c1954f32b15a5fbdc52511b9
59f5fb0ea20840878cab45f8e1eb2c0f
d606e50af4bf49fbb3344e666b3a0d69
e44e5a84f8034db1a9fab5b391c2cd34


In [70]:
import requests
URL = 'http://localhost:5000/entries/e44e5a84f8034db1a9fab5b391c2cd35'
x = requests.get(URL)
x.text

'<!DOCTYPE html>\n<html lang="en">\n    <head>\n        <meta charset="utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1">\n        <link rel="stylesheet"\n              href="https://cdn.jsdelivr.net/npm/bootstrap@4.1.3/dist/css/bootstrap.min.css"\n              integrity="sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO"\n              crossorigin="anonymous">\n        <link rel="stylesheet"\n              type="text/css"\n              href="/static/prism.css">\n        <link rel="stylesheet"\n              type="text/css"\n              href="/static/style.css">\n        <title>DFT-CODEX</title>\n        <meta charset="utf-8" />\n        <meta name="keywords" content="dft,vasp,quantum-espresso" />\n        <meta name="description"\n              content="Density Functional Theory Comprehensive Documentation Explorer" />\n    </head>\n    <body>\n        <div class="container">\n            <h1 class="display-1">DFT-CODEX</h1>

In [81]:
result = client['codex']['entries'].delete_one({'_id': 'dff217b9190e4d1db5d118d09d0463f7'})

In [88]:
URL = 'http://localhost:5000/entries/d606e50af4bf49fbb3344e666b3a0d69'
requests.delete(URL)

<Response [204]>

In [85]:
import requests
URL = 'http://localhost:5000/entries'
#from codex.api.schemas import CodexEntryQueryArgsSchema
#print(CodexEntryQueryArgsSchema().load(params))
params = {'filetype': 'INCAR'}
x = requests.get(URL, params=params).json()
for xx in x:
    print(xx['_id'])

3954583e-94ba-41df-90f3-86d673ce5189
83e594ee-ad85-4c81-b72f-ffa53d8ce9a4
4b364556c1954f32b15a5fbdc52511b9
59f5fb0ea20840878cab45f8e1eb2c0f
d606e50af4bf49fbb3344e666b3a0d69
e44e5a84f8034db1a9fab5b391c2cd34


In [37]:
import requests
from codex.models.vasp_codex import VaspCodex
from codex.models.espresso_codex import EspressoCodex
from marshmallow import Schema, fields, validate, ValidationError, pre_dump, pre_load
from codex.api.schemas import CodexEntrySchema

codex = VaspCodex('example/vasp/INCAR_HfTe5.vasp', 'latest', client)
codex1 = VaspCodex('example/vasp/POSCAR_Fe3O4.vasp', 'latest', client)
codex2 = EspressoCodex('example/espresso/bi4te3_bandsx.in', 'latest', client)
data = CodexEntrySchema().dump([codex], many=True)
#print(data.keys())
#double_dump = CodexEntrySchema().dump(data)
#double_dump.keys()

# Add to MongoDB

In [ ]:
base_db_dir = resources.files("codex.database")

json_files = glob.glob(os.path.join(base_db_dir, "json", "*.json"))
for file in json_files:
    insert_json(client, file)

In [ ]:
database_names = client.list_database_names()
available_versions = [
    match.group(1)
    for f in database_names
    for match in [re.match(rf"{code}-([0-9]+\^[0-9]+(?:\^[0-9]+)?)", f)]
    if match
]
print(available_versions)

In [ ]:
get_database(client, 'Quantum ESPRESSO', '7.1')

# VASP database

In [ ]:
db = vasp_generate_json()

# QE database

In [ ]:
versions = ['6.7', '6.8', '7.0', '7.1', '7.2']
database_dir = resources.files("codex.database")

for v in versions:
    #run_helpdoc(v)
    db = espresso_generate_json(v)

# Generate Codex

In [ ]:
from importlib import resources
filename = 'example/sr3pbo_bands.in'
database_dir = resources.files("codex.database")
version = '7.2'
# Make .dft-tutor directory in current dir if it doesn't exist
codex = Codex(filename, version)


In [ ]:
print(codex.filename)
print(codex.code)
print(codex.package)
print(codex.tags['control'])

In [ ]:
from importlib import resources
import os
from base64 import b64decode
import json
import shutil

dbversion='7.2'
dir='static'

base_db_dir = resources.files("codex.database")
code = "qe"  # Need better terminology
database_dir = os.path.join(base_db_dir, f"{code}-{dbversion}")
database_filename = os.path.join(database_dir, "database.json")

with open(database_filename) as f:
    database = json.load(f)

working_dir = os.path.abspath(dir)
tags_dir = os.path.join(working_dir, "tags-qe")
if os.path.exists(tags_dir):
    shutil.rmtree(tags_dir, ignore_errors=True)
os.mkdir(tags_dir)
for p, package in database.items():
    print(p)
    os.mkdir(os.path.join(tags_dir, p))
    for nl, namelist in package.items():
        print(nl)
        os.mkdir(os.path.join(tags_dir, p, nl))
        for tag, v in namelist.items():
            print(tag, v)
            if v["html"] != 'No documentation was found for this tag.':
                webpage = b64decode(v["html"]).decode("utf-8")
                path = os.path.join(tags_dir, p, nl, f"{tag}.html")
                with open(path, "w") as f:
                    f.write(webpage)